In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Data Preparation

## Combining Data

In [6]:
def combining_data():
    all_stadiums = pd.DataFrame([], columns=['stadium_name', 'href', 'city', 'capacity', 'key'])
    for filename in os.listdir('datasets/stadiums'):
        stadiums = pd.read_csv('datasets/stadiums/{}'.format(filename))
        stadiums.drop(['Unnamed: 0'], axis=1, inplace=True)
        season = filename.split('_')[0]
        stadiums['key'] = stadiums['href'].apply(lambda v: season + '_' + v.split('/')[2])
        all_stadiums = pd.concat([all_stadiums, stadiums])

    all_stadiums = pd.DataFrame(all_stadiums)
    all_matches = pd.DataFrame([], columns=['match_type', 'date', 'home_team', 'away_team', 'result','key'])
    for i in os.listdir('datasets/matches'):
        for filename in os.listdir('datasets/matches/%s' % i ):
            matches = pd.read_csv(('datasets/matches/%s/{}' % i).format(filename))
            matches.drop(['Unnamed: 0'], axis=1, inplace=True)
            season = filename.split('_')[0]
            matches['key'] = '%s' % i + '_' + season
            all_matches = pd.concat([all_matches, matches])

    all_matches = pd.DataFrame(all_matches)

    all = pd.merge(all_matches, all_stadiums, how='left', on='key')

    all_select =['match_type', 'date', 'home_team', 'away_team', 'result','stadium_name', 'capacity_x','capacity_y']
    all_select = all[all_select]

    all_select = all_select.query('match_type == "Pr. League"')
    all_select.drop_duplicates(subset=['date','home_team','away_team'],keep='first', inplace=True) #delete repeat rows
    all_select.replace('???',"0", inplace=True) 

    all_select['capacity_x'] = pd.to_numeric(all_select['capacity_x'])
    all_select['capacity_y'] = pd.to_numeric(all_select['capacity_y'])
    all_select['Attendance'] = all_select['capacity_x'] / all_select['capacity_y'] 
    all_select.to_csv('datasets/raw_data.csv')

combining_data()

In [7]:
pd.read_csv('datasets/raw_data.csv').head()

,Unnamed: 0,match_type,date,home_team,away_team,result,stadium_name,capacity_x,capacity_y,Attendance
0,1,Pr. League,13/01/2018,newcastle-united,swansea-city,\n1:1 \n,St James' Park,51.444,52.409,0.981587
1,2,Pr. League,31/01/2018,newcastle-united,burnley-fc,\n1:1 \n,St James' Park,50.174,52.409,0.957355
2,3,Pr. League,11/02/2018,newcastle-united,manchester-united,\n1:0 \n,St James' Park,52.309,52.409,0.998092
3,4,Pr. League,10/03/2018,newcastle-united,southampton-fc,\n3:0 \n,St James' Park,52.246,52.409,0.996890
4,5,Pr. League,31/03/2018,newcastle-united,huddersfield-town,\n1:0 \n,St James' Park,52.261,52.409,0.997176


## Cleansing

In [8]:
def data_cleansing():
    # Read data
    raw=pd.read_csv('datasets/raw_data.csv')

    temp=raw.copy()
    temp.drop(columns=['Unnamed: 0'],inplace=True) # delete reduent index col

    # rename
    temp=temp.rename(columns={
        'stadium_name':'Stadium',
        'capacity_x':'Attendance',
        'Attendance':'Attendance_Percentage',
        'capacity_y':'Capacity',
        'match_type':'Match_Type',
        'date':'Date',
        'home_team':'Home_Team',
        'away_team':'Away_Team',
        'result':'Result',
    })

    # edit Date, Attendance and Capacity
    date=temp['Date'].tolist()
    for i in range(len(date)):
        t=date[i].split('/')
        date[i]=datetime(int(t[2]),int(t[1]),int(t[0]))
    temp['Date']=date
    temp['Attendance']=temp['Attendance']*1000
    temp['Capacity']=temp['Capacity']*1000


    # add season column
    start=datetime(1992,7,31)
    temp1=temp[temp['Date']>start]
    temp2=temp1.sort_values('Date',ascending=True)
    l=[]
    for i in range(30):
        year=1992+i
        start=datetime(year,8,1)
        end=datetime(year+1,7,31)
        season=temp1[(temp1['Date']>start)&(temp1['Date']<end)]
        lt=['%d-%d' %(year,year+1)]*(len(season))
        l=l+lt
    temp2['Season']=l

    temp2.reset_index(drop=True, inplace=True)
    temp3 = temp2.copy()

    # add Result, Win, Score, Lost column
    Win=[]
    Lost=[]
    Score=[]
    Result=[]
    for i in range(11510):
        a=temp3.loc[i]['Result']
        b=a.split()
        c=b[0].split(":")
        if c[0]=='-':
            Win.append(None)
            Score.append(None)
            Lost.append(None)
            Result.append(None)
            continue
        score=int(c[0])
        lost=int(c[1])
        Score.append(score)
        Lost.append(lost)
        Result.append(b[0])
        if score>=lost:
            Win.append(1)
        else:
            Win.append(0)
    temp3['Result']=Result+[None]*136
    temp3['Win']=Win+[None]*136
    temp3['Score']=Score+[None]*136
    temp3['Lost']=Lost+[None]*136

    temp4=temp3.copy()
    # changing attendance to 2000 due to the policy: https://www.premierleague.com/news/1924351
    condition1 = (temp3['Date'] >= np.datetime64('2020-12-05')) & (temp3['Date'] < np.datetime64('2021-05-17'))
    temp4.loc[condition1, 'Attendance'] = 2000
    temp4['Attendance_Percentage']=temp4['Attendance']/temp4['Capacity']

    temp4['Date'] = temp4['Date'].astype(str)
    temp4['Date'] = temp4['Date'].str.replace('-', '/')

    # after checking source, we found manchester stadium size is wrong, we correct using the current size
    temp4.loc[((temp4['Home_Team'] == "manchester-united")), 'Capacity'] = 76212

    # capacity wrong in west-ham-united: https://www.southamptonfc.com/matches/2017-18/west-ham-v-saints-2018-03-31
    # temp3.query('Home_Team ==  & Away_Team == "" & Date == ')
    temp4.loc[(
        (temp4['Home_Team'] == "west-ham-united") & 
        (temp4['Away_Team'] == 'southampton-fc') &
        (temp4['Date'] =="2018/03/31")
    ), 'Capacity'] = 60000

    # fulham-fc,Loftus Road, including standing ticket, https://en.wikipedia.org/wiki/Loftus_Road

    # Middlesbrough capacity change if necessary: https://en.wikipedia.org/wiki/Riverside_Stadium#:~:text=The%20Riverside%20Stadium%20is%20a,that%20to%2042%2C000%20if%20required.
    # middlesbrough_exception_query = (temp4['Home_Team'] == 'middlesbrough-fc') & (temp4['Attendance_Percentage'] > 1)
    # middlesbrough_exception = temp4[middlesbrough_exception_query].copy()
    # temp4.loc[middlesbrough_exception_query, 'Capacity'] = middlesbrough_exception['Attendance']

    # add played column
    n=np.zeros(11646)
    year=1992
    for k in range(30):
        current='"'+str(year)+"-"+str(year+1)+'"'
        teams='Season==%s' %(current)
        team=temp4.query(teams)
        u=team["Home_Team"].unique().tolist()
        for i in u:
            club='"'+i+'"'
            sentence='(Season==%s) and (Home_Team==%s or Away_Team==%s)' %(current,club,club)
            games=temp4.query(sentence)
            Index=games.index.to_list()
            for j in range(len(Index)):
                n[Index[j]]=j+1
        year=year+1
    temp4['Played']=n
    temp4[temp4["Played"]==0]

    temp4.to_csv('datasets/raw_data3.csv')
data_cleansing()

### Transformation

In [9]:
def transform_data():
    # feature x3-x6
    review=pd.read_csv('datasets/raw_data3.csv')

    def ComputePoint(score,lost):
        if score>lost:
            return 3
        elif score==lost:
            return 1
        else:
            return 0

    def SeasonRanking(temp):
        start=1992
        rank=pd.DataFrame(columns=('Club','Season','Points','GF','GA'))
        for i in range(29):
            current=str(start)+"-"+str(start+1)
            Games=temp[temp['Season']==current]
            clubs=Games['Home_Team'].unique().tolist()
            for club in clubs:
                points=0
                home=Games.query('Home_Team==%s' %('"'+club+'"')).reset_index(drop=True)
                away=Games.query('Away_Team==%s' %('"'+club+'"')).reset_index(drop=True)
                gf=np.sum(home['Score'])+np.sum(away['Lost'])
                ga=np.sum(home['Lost'])+np.sum(away['Score'])
                home['Point']=home.apply(lambda x: ComputePoint(x['Score'],x['Lost']),axis=1)
                away['Point']=away.apply(lambda x: ComputePoint(x['Lost'],x['Score']),axis=1)
                points=np.sum(home['Point'])+np.sum(away['Point'])
                rank=rank.append({'Club':club,'Points':points,'GF':gf,'GA':ga,'Season':current},ignore_index=True)
            start=start+1
        rank['GD']=rank['GF']-rank['GA']
        return rank

    def RankingOfLastSeason(temp):
        rank=SeasonRanking(temp)
        temp1=temp.copy()
        temp1['Ranking_Of_Last_Season_Home']=0
        temp1['Ranking_Of_Last_Season_Away']=0
        start=1993
        for i in range(29):
            last=str(start-1)+"-"+str(start)
            current=str(start)+"-"+str(start+1)
            ranking=rank[rank['Season']==last].sort_values(by=['Points','GD','GF'],ascending=False).reset_index(drop=True)
            ranking['Ranking']=ranking.index+1
            game=temp1[temp1['Season']==current]
            clubs=game['Home_Team'].unique().tolist()
            for c in clubs:
                l=list(ranking[ranking['Club']==c]['Ranking'])
                if l:
                    temp1.loc[game[game['Home_Team']==c].index,'Ranking_Of_Last_Season_Home']=l[0]
                    temp1.loc[game[game['Away_Team']==c].index,'Ranking_Of_Last_Season_Away']=l[0]
            start=start+1
        temp1.loc[temp1[temp1['Ranking_Of_Last_Season_Away']==0].index,'Ranking_Of_Last_Season_Away']=20
        temp1.loc[temp1[temp1['Ranking_Of_Last_Season_Home']==0].index,'Ranking_Of_Last_Season_Home']=20
        return temp1

    temp2=RankingOfLastSeason(review)
    def CurrentRanking2(temp):
        temp2=temp.copy()
        seasons = temp2['Season'].unique()
        for Season in seasons:
            Games=temp2[temp2['Season']==Season].copy()
            clubs=Games['Home_Team'].unique()
            Games['Index']=Games.index
        
            Playeds = np.sort(Games['Played'].unique())
            for j in Playeds:
                if j == 0:
                    continue
                ranking=pd.DataFrame(columns=('Club','Played','Points','GF','GA'))
                played=Games[Games['Played']<j]
                for club in clubs:
                    gf=0
                    ga=0
                    points=0
                    home=played[played['Home_Team'] == club].copy()
                    away=played[played['Away_Team'] == club].copy()
                    
                    if home['Home_Team'].any():
                        gf=gf+np.sum(home['Score'])
                        ga=ga+np.sum(home['Lost'])
                        home['Point']= home.apply(lambda x: ComputePoint(x['Score'],x['Lost']),axis=1)
                        points=points+np.sum(home['Point'])
                    if away['Away_Team'].any():
                        gf=gf+np.sum(away['Lost'])
                        ga=ga+np.sum(away['Score'])
                        away['Point']=away.apply(lambda x: ComputePoint(x['Lost'],x['Score']),axis=1)
                        points=points+np.sum(away['Point'])
                    ranking=ranking.append({'Club':club,'Points':points,'GF':gf,'GA':ga,'Played':j-1},ignore_index=True)
                    
                ranking['GD']=ranking['GF']-ranking['GA']
                ranking=ranking.sort_values(by=['Points','GD','GF'],ascending=False).reset_index(drop=True)
                ranking['Ranking']=ranking.index+1
                playing=Games[Games['Played']==j]
                for k in range(len(playing)):
                    # print(Season, j, k, playing.iloc[k]['Home_Team'])
                    ranking_home=list(ranking[ranking['Club']==playing.iloc[k]['Home_Team']]['Ranking'])[0]
                    temp2.loc[playing.iloc[k]['Index'],'Ranking_Home']=ranking_home
                    ranking_away=list(ranking[ranking['Club']==playing.iloc[k]['Away_Team']]['Ranking'])[0]
                    temp2.loc[playing.iloc[k]['Index'],'Ranking_Away']=ranking_away

        return temp2

    temp3 = CurrentRanking2(temp2)


    # feature x7 - x16
    add=pd.read_csv('datasets/raw_data3.csv')
    add[['X7','X8','X9','X10','X11','X12','X13','X14','X15','X16','X17']] = ''

    def earn(win,lost):
        if win>lost:
            return 4
        elif win==lost:
            return 2
        elif win<lost:
            return 1
        else:
            return 0

    def check_before(x,team):
        if x['Home_Team'].values==team:
            return earn(x['Score'].values,x['Lost'].values)
        else:
            return earn(x['Lost'].values,x['Score'].values)
            
    def check_list(i,team,season,time):
        check=add[(add['Season'] ==season) & ((add['Home_Team'] == team) | (add['Away_Team'] == team))].reset_index()
        check['Date'] = pd.to_datetime(check['Date'], format='%Y/%m/%d')
        before=check[check['Date']<time]
        l=len(before)
        if l<i:
            return 0
        else:
            return check_before(before.loc[l-i:l-i,:],team)
            
    def home_5():
        for i in range(1,6):
            n='X'+str(i+6)
            add[n]=add.apply(lambda v: check_list(i,v['Home_Team'],v['Season'],v['Date']), axis=1)
    def away_5():
        for j in range(1,6):
            n='X'+str(j+11)
            add[n]=add.apply(lambda v: check_list(j,v['Away_Team'],v['Season'],v['Date']), axis=1)

    home_5()
    away_5()


    # x17-x20
    raw=pd.read_csv('datasets/raw_data3.csv')
    data = raw.copy()
    data.drop('Unnamed: 0', inplace=True, axis=1)
    data['Datetime'] = pd.to_datetime(data['Date'], format='%Y/%m/%d')

    def get_sum_score(data, series, n=1, col='Home_Team', key='Score'):
        """
        Summary previous games' features by the team_name in the specific season

        Parameters:
        data: whole data which type is DataFrame 
        series: the specific row in the data
        n: how many last previous rows you want to invole
        col: Home_Team or Away_Team
        key: the numeric feature you want to sum
        """
        date = series['Datetime']
        season = series['Season']
        team_name = series[col]

        sum_score = 0
        data_filtered = data[(data[col] == team_name) & (data['Season'] == season)]
        data_filtered.sort_values(['Datetime'], inplace=True)
        data_filtered = data_filtered[data_filtered['Datetime']<date]
        last_n_rows = data_filtered.iloc[-n:, :]
        return last_n_rows['Score'].sum()


    def x17_x20(raw):
        data = raw.copy()

        # X17: Home_Team + Score
        data['X17'] = data.apply(lambda v: get_sum_score(data, v, 5, 'Home_Team', 'Score'), axis=1)
        # X18: Away_Team + Score
        data['X18'] = data.apply(lambda v: get_sum_score(data, v, 5, 'Away_Team', 'Score'), axis=1)
        # X19: Home_Team + Lost
        data['X19'] = data.apply(lambda v: get_sum_score(data, v, 5, 'Home_Team', 'Lost'), axis=1)
        # X20: Away_Team + Lost
        data['X20'] = data.apply(lambda v: get_sum_score(data, v, 5, 'Away_Team', 'Lost'), axis=1)

        return data

    Features17to20 = x17_x20(data)


    # x21
    data=pd.read_csv('datasets/raw_data3.csv')
    data['Result'] = data['Result'].astype(str)
    data['Result1']=[x[:1] for x in data['Result']]
    data['Result2']=[x[-1:] for x in data['Result']]
    data = data[data.Result1 != 'n']
    data['Result1'] = pd.to_numeric(data['Result1'])
    data = data[data.Result2 != 'n']
    data['Result2'] = pd.to_numeric(data['Result2'])
    data['origin_Date'] = data['Date']
    data['Date'] = pd.to_datetime(data['Date'], format='%Y/%m/%d')
    groups = data.groupby(['Home_Team','Away_Team']) 
    data_X21 =pd.DataFrame()
    for group in groups:
        home_key, away_key = group[0]
        a = data.query('Home_Team == "%s"'% home_key)
        b = a.query('Away_Team == "%s"'% away_key)

        c = data.query('Home_Team == "%s"'% away_key)
        d = c.query('Away_Team == "%s"'% home_key)

        df = pd.concat([b,d])
        df1 = df.sort_values('Date', ascending=True)
        df1_sort = df1.reset_index(drop= True)

        date2_sort = b.sort_values('Date', ascending=True)
        date2_sort = date2_sort.reset_index(drop= True)

        for i in range(len(date2_sort)):
            row = date2_sort.iloc[i] 
            select_row = df1_sort.loc[lambda df1_sort:(df1_sort['Date'] < row['Date']) ,:]
            select_4 = min(4,len(select_row))
            select_4row = select_row.iloc[-select_4:, :]

            X21_score = 0
            for k in range(len(select_4row)):
                r1 = select_4row.iloc[k]['Result1']
                r2 = select_4row.iloc[k]['Result2']
                if select_4row.iloc[k].at['Home_Team'] == row['Home_Team'] and r1 > r2:
                    X21_score = X21_score + 4
        
                elif select_4row.iloc[k].at['Home_Team'] != row['Home_Team'] and r1 < r2:
                    X21_score = X21_score + 4

                elif r1 == r2:
                    X21_score = X21_score + 2

                elif select_4row.iloc[k].at['Home_Team'] == row['Home_Team'] and r1 < r2:
                    X21_score = X21_score + 1
                
                elif select_4row.iloc[k].at['Home_Team'] != row['Home_Team'] and r1 > r2:
                    X21_score = X21_score + 1
            
            row['X21'] = X21_score
            data_X21 = data_X21.append(row)


    # integrating
    # read raw data
    features = pd.read_csv('datasets/raw_data3.csv')
    features.set_index(['Home_Team', 'Away_Team', 'Date'], inplace=True)
    features.head()

    # configuration
    features_raw_name = {
        'x3-x6': ['Ranking_Of_Last_Season_Home', 'Ranking_Of_Last_Season_Away',
        'Ranking_Home', 'Ranking_Away'],
        'x7-x16': ['X7', 'X8', 'X9', 'X10', 'X11', 'X12', 'X13', 'X14', 'X15', 'X16'],
        'x17-x20':['X17', 'X18', 'X19', 'X20'],
        'x21':['X21']
    }

    # func
    def conbine_features(features, processed_data, cols, index_cols=['Home_Team', 'Away_Team', 'Date']):
        processing_data = processed_data.set_index(index_cols)
        return pd.concat([features, processing_data[cols]], axis=1)

    # read data
    features = conbine_features(features, temp3, features_raw_name['x3-x6'])
    # print(features.shape, temp3.shape)
    features = conbine_features(features, add, features_raw_name['x7-x16'])
    # print(features.shape, add.shape)
    features = conbine_features(features, Features17to20, features_raw_name['x17-x20'])
    # print(features.shape)
    features = conbine_features(features, data_X21, features_raw_name['x21'], ['Home_Team', 'Away_Team', 'origin_Date'])
    # print(features.shape)



    # Rename columns
    calculated_features_name = pd.concat([pd.DataFrame(features_raw_name[key]) for key in features_raw_name])[0].to_list()
    print(calculated_features_name)

    reset_features = features.reset_index()

    name_maps = {
        'Home_Team':'HomeTeam',
        'Away_Team':'AwayTeam',
        'level_2':'Date',
        'Season':'Season',
        'Capacity':'X1',
        'Attendance_Percentage':'X2',
        'Ranking_Home':'X3',
        'Ranking_Away':'X4',
        'Ranking_Of_Last_Season_Home':'X5',
        'Ranking_Of_Last_Season_Away':'X6',
        'X7':'X7',
        'X8':'X8',
        'X9':'X9',
        'X10':'X10',
        'X11':'X11',
        'X12':'X12',
        'X13':'X13',
        'X14':'X14',
        'X15':'X15',
        'X16':'X16',
        'X17':'X17',
        'X18':'X18',
        'X19':'X19',
        'X20':'X20',
        'X21':'X21',
        'Played':'X22',
        'Win':'Y'
    }

    filtering_name_map = {}

    for name in name_maps:
        if name_maps[name]!= '':
            filtering_name_map[name] = name_maps[name]

    renamed_features = reset_features.rename(columns = filtering_name_map)

    # refactor Date value
    renamed_features['Date'] = pd.to_datetime(renamed_features['Date'], format='%Y/%m/%d')
    renamed_features.sort_values('Date', inplace=True)
    renamed_features['Date'] = renamed_features['Date'].dt.strftime('%Y/%m/%d')

    # get specific columns
    final_features = renamed_features[list(name_maps.values())]
    checking_features = final_features.set_index(['HomeTeam', 'AwayTeam', 'Date'])
    # print()
    def check_features_by_season(feature):
        featureWithNull = checking_features[checking_features[feature].isnull()]
        print(feature, featureWithNull['Season'].unique())

    # check null value
    print('if column have none value:')
    check_features_by_season(name_maps['X7'])
    check_features_by_season(name_maps['X8'])
    check_features_by_season(name_maps['X9'])
    check_features_by_season(name_maps['X10'])
    check_features_by_season(name_maps['X11'])
    check_features_by_season(name_maps['X12'])
    check_features_by_season(name_maps['X13'])
    check_features_by_season(name_maps['X14'])
    check_features_by_season(name_maps['X15'])
    check_features_by_season(name_maps['X16'])
    check_features_by_season(name_maps['X21'])

    # Filter data
    # 1. delete season 1992-1993, cause the feature about last season ranking doesn't have data.
    # 2. delete part of the sesson data 2021-2022, cause part of games don't exist results.
    features_filtered = final_features[final_features['Season'] != '1992-1993']
    features_filtered.dropna(inplace=True, axis=0)
    features_filtered.to_csv('datasets/final.csv', index=False)

transform_data()

['Ranking_Of_Last_Season_Home', 'Ranking_Of_Last_Season_Away', 'Ranking_Home', 'Ranking_Away', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21']
if column have none value:
X7 []
X8 []
X9 []
X10 []
X11 []
X12 []
X13 []
X14 []
X15 []
X16 []
X21 ['2021-2022']
